In [2]:
ls = sc.textFile('/home/dsc/Data/spark/coupon150720.csv')

In [2]:
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [3]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [4]:
ls.count()

1232662

In [5]:
ls.cache()

MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [6]:
ls.count()

1232662

In [7]:
ls.count()

1232662

In [8]:
ls.getNumPartitions()

3

In [9]:
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

### Número de pasajeros operados por Iberia

#### Cada línea es un pasajero. El código del operador es la 6ª columna. El código de Iberia es IB

In [11]:
ls.filter(lambda x: x.split(',')[5]=="IB").count()

26158

In [33]:
kv = ls.map(lambda x: x.split(',')[5])

In [36]:
kv.filter(lambda y: y=="IB").count()

26158

In [37]:
# En una linea
ls.map(lambda x: x.split(',')[5]).filter(lambda y: y=="IB").count()

26158

### Cuanto ha ganado Iberia con estos vuelos?

#### El dinero es la 7ª columna

In [35]:
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])
    return (aerolinea, dinero)

In [36]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [42]:
getAerolineaDinero

<function __main__.getAerolineaDinero>

In [37]:
dineroIB = ls.map(getAerolineaDinero).filter(lambda x: x[0] =="IB")

In [38]:
cantidades = dineroIB.map(lambda x: x[1]).sum()

In [30]:
cantidades

2828044.4199999194

In [76]:
totales = dineroIB.map(lambda x: x[1])

In [77]:
totales.stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

### Top ten de rutas con billetes mas caros

In [54]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [62]:
def getrutadinero(l):
    elems = l.split(",")
    origen = elems[2]
    dest = elems[3]
    precio = float(elems[6])
    return (origen + '-' + dest, precio)

In [63]:
lista = ls.map(getrutadinero)

In [65]:
lista.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [68]:
lista.filter(lambda x: x[1]<7500).sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [76]:
listagroup = lista.filter(lambda x: x[1]<7500).groupByKey().mapValues(lambda vs: max(vs))

In [77]:
listagroup.sortBy(lambda y: y[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

In [78]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [81]:
def getRutaDinero(l):
    elems = l.split(',')
    origen = elems[2]
    destino = elems[3]
    dinero = float(elems[6])
    ruta = origen + "-" + destino
    return (ruta, dinero)

In [82]:
getRutaDinero(ls.first())

(u'MAA-AUH', 56.79)

In [83]:
rutas = ls.map(getRutaDinero)

In [84]:
rutas.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [85]:
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [86]:
filtrado = rutas.filter(lambda x: x[1] < 7500)

In [90]:
filtrado.cache()

PythonRDD[80] at RDD at PythonRDD.scala:43

In [87]:
filtrado.groupByKey().take(3)

[(u'SZF-IST', <pyspark.resultiterable.ResultIterable at 0x7ffb222688d0>),
 (u'DME-RHO', <pyspark.resultiterable.ResultIterable at 0x7ffb22268050>),
 (u'KRK-OSL', <pyspark.resultiterable.ResultIterable at 0x7ffb222680d0>)]

In [91]:
filtrado.groupByKey().mapValues(lambda vs: max(vs)).take(3)

[(u'SZF-IST', 289.83), (u'DME-RHO', 168.95), (u'KRK-OSL', 98.0)]

In [92]:
maximos = filtrado.groupByKey().mapValues(lambda vs: max(vs))

In [93]:
maximos.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

### Ruta con el valor medio por cupon más alto (Top 5)

In [97]:
filtrado.groupByKey().mapValues(lambda vs: sum(vs)/len(vs)).sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

### Eliminar outliers Aerolinea Ruta y Booking Class (campos 2,3,4,6 y 9)

#### Calculamos la media del logaritmo del precio. Despreciamos los que son mayores que 2 veces la desviacion estándar

In [100]:
def getRutaALDineroClase(l):
    elems = l.split(",")
    ruta = elems[2] + "-" + elems[3]
    al = elems[4]
    dinero = float(elems[6])
    clase = elems[9]
    return ((ruta, al, clase), dinero)

In [101]:
getRutaALDineroClase(ls.first())

((u'MAA-AUH', u'9W', u'H'), 56.79)

In [102]:
import math

In [127]:
def valorLimite(vs):
    logs = [math.log(v+1) for v in vs]
    #vs.map(lambda x: math.log(x))
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    var = sum(desv)/len(desv)
    desvStd = math.sqrt(var)
    return math.exp(media + 2*desvStd) - 1

In [128]:
valorLimite([12,3,4,1,2,4,115])

86.96516439844903

In [129]:
rutas = ls.map(getRutaALDineroClase)

In [130]:
rutas.take(5)

[((u'MAA-AUH', u'9W', u'H'), 56.79),
 ((u'AUH-CDG', u'9W', u'H'), 84.34),
 ((u'CJB-MAA', u'9W', u'H'), 60.0),
 ((u'DEL-DXB', u'9W', u'S'), 160.63),
 ((u'AUH-IXE', u'9W', u'V'), 152.46)]

In [131]:
rutas.groupByKey().take(2)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7ffb222683d0>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7ffb22311e50>)]

In [132]:
def filtraCuponesExtermos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v < vl]
    return f

In [133]:
grupos = rutas.groupByKey().mapValues(filtraCuponesExtermos)

In [138]:
def mediaGrupo(vs):
    if (len(vs)==0):
        return 0.0
    else:
        return sum(vs)/len(vs)

In [140]:
grupos.mapValues(mediaGrupo).sortBy(lambda x: x[1], False).take(5)

[((u'SYD-CGK', u'GA', u'G'), 6355194.0),
 ((u'COO-CDG', u'AF', u'J'), 538599.2),
 ((u'CDG-COO', u'AF', u'J'), 179628.3533333333),
 ((u'DKR-NBO', u'KQ', u'T'), 58253.593333333345),
 ((u'TSN-ICN', u'KE', u'Z'), 38655.45)]

In [141]:
grupos.take(2)

[((u'TRD-BOO', u'SK', u'V'),
  [42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   48.67,
   42.76,
   48.67,
   42.76,
   42.76,
   42.76,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76,
   29.55,
   42.76,
   42.76]),
 ((u'DPS-TMC', u'GA', u'N'), [59.15, 59.26, 62.38, 62.38])]

In [142]:
grupos = rutas.groupByKey()

In [143]:
grupos.take(5)

[((u'TRD-BOO', u'SK', u'V'),
  <pyspark.resultiterable.ResultIterable at 0x7ffb222cb710>),
 ((u'DPS-TMC', u'GA', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7ffb222cbe10>),
 ((u'BEG-CDG', u'JU', u'K'),
  <pyspark.resultiterable.ResultIterable at 0x7ffb222cbb10>),
 ((u'SAV-CLT', u'US', u'N'),
  <pyspark.resultiterable.ResultIterable at 0x7ffb222cb450>),
 ((u'DUS-AMS', u'KL', u'C'),
  <pyspark.resultiterable.ResultIterable at 0x7ffb222cb810>)]

In [145]:
r = grupos.filter(lambda x: len(x[1]) > 10).mapValues(filtraCuponesExtermos).mapValues(mediaGrupo)

In [146]:
r.take(2)

[((u'TRD-BOO', u'SK', u'V'), 42.30263157894736),
 ((u'PHX-CLT', u'US', u'S'), 157.65285714285713)]

In [147]:
r.sortBy(lambda x: x[1], False).take(5)

[((u'MIA-CCS', u'S3', u'S'), 7826.25),
 ((u'CCS-MIA', u'S3', u'B'), 5550.0),
 ((u'CCS-MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'RUH-FRA', u'LH', u'F'), 4792.983),
 ((u'LHR-DOH', u'QR', u'F'), 4275.98923076923)]

### Ver cuantos valores diferentes hay en un dataset

In [149]:
ls.map(lambda x: x.split(",")[4]).distinct().count()

359

In [150]:
ls.map(lambda x: x.split(",")[7]).distinct().collect()

[u'USD']

### Grabar datos en ficheros

In [ ]:
ls.map(lambda x: x.split(",")[4]).distinct().saveAsTextFile("results.csv")